In [1]:
"""
Utilities used by our other RNN scripts.
"""
from collections import deque
from sklearn.cross_validation import train_test_split
from tflearn.data_utils import to_categorical
import tflearn
import tensorflow as tf
import numpy as np
import pickle

def get_data(filename, num_frames, num_classes, input_length):
    """Get the data from our saved predictions or pooled features."""

    # Local vars.
    X = []
    y = []
    temp_list = deque()

    # Open and get the features.
    with open(filename, 'rb') as fin:
        frames = pickle.load(fin)

        for i, frame in enumerate(frames):
            features = frame[0]
            actual = frame[1]

            # Convert our labels into binary.
            if actual == 'again':
                actual = 0
            elif actual == 'cool':
                actual = 1
            elif actual == 'dont know':
                actual = 2
            elif actual == 'fingerspell':
                actual = 3
            elif actual == 'from':
                actual = 4
            elif actual == 'have to':
                actual = 5
            elif actual == 'her his':
                actual = 6
            elif actual == 'their':
                actual = 7
            elif actual == 'why':
                actual = 8
            elif actual == 'your':
                actual = 9

            # Add to the queue.
            if len(temp_list) == num_frames - 1:
                temp_list.append(features)
                flat = list(temp_list)
                X.append(np.array(flat))
                y.append(actual)
                temp_list.popleft()
            else:
                temp_list.append(features)
                continue

    print("Total dataset size: %d" % len(X))
    # Numpy.
    X = np.array(X)
    print(X.shape)
    
    y = np.array(y)
    print(y.shape)

    # Reshape.
    X = X.reshape(-1, num_frames, input_length)

    # One-hot encoded categoricals.
    y = to_categorical(y, num_classes)

    # Split into train and test.
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.1, random_state=42)

    return X_train, X_test, y_train, y_test

def get_network(frames, input_size, num_classes):
    """Create our LSTM"""
    net = tflearn.input_data(shape=[None, frames, input_size])
    net = tflearn.lstm(net, 128, dropout=0.8, return_seq=True)
    net = tflearn.lstm(net, 128)
    net = tflearn.fully_connected(net, num_classes, activation='softmax')
    net = tflearn.regression(net, optimizer='adam',
                             loss='categorical_crossentropy', name="output1")
    return net

def get_network_deep(frames, input_size, num_classes):
    """Create a deeper LSTM"""
    net = tflearn.input_data(shape=[None, frames, input_size])
    net = tflearn.lstm(net, 64, dropout=0.2, return_seq=True)
    net = tflearn.lstm(net, 64, dropout=0.2, return_seq=True)
    net = tflearn.lstm(net, 64, dropout=0.2)
    net = tflearn.fully_connected(net, num_classes, activation='softmax')
    net = tflearn.regression(net, optimizer='adam',
                             loss='categorical_crossentropy', name="output1")
    return net

def get_network_wide(frames, input_size, num_classes):
    """Create a wider LSTM"""
    net = tflearn.input_data(shape=[None, frames, input_size])
    net = tflearn.lstm(net, 256, dropout=0.2)
    net = tflearn.fully_connected(net, num_classes, activation='softmax')
    net = tflearn.regression(net, optimizer='adam',
                             loss='categorical_crossentropy', name='output1')
    return net

def get_network_wider(frames, input_size, num_classes):
    """Create a wider LSTM"""
    net = tflearn.input_data(shape=[None, frames, input_size])
    net = tflearn.lstm(net, 512, dropout=0.2)
    net = tflearn.fully_connected(net, num_classes, activation='softmax')
    net = tflearn.regression(net, optimizer='adam',
                             loss='categorical_crossentropy', name='output1')
    return net

In [2]:
def train_main(filename, frames, batch_size, num_classes, input_length):
    """From the blog post linked above."""
    # Get our data.

    X_train, X_test, y_train, y_test = get_data(filename, frames, num_classes, input_length)

    # Get sizes.
    num_classes = len(y_train[0])

    tf.reset_default_graph()
    # Get our network.
    net = get_network_wide(frames, input_length, num_classes)

    # Train the model.
    model = tflearn.DNN(net, tensorboard_verbose=0)
    
    model.fit(X_train, y_train, validation_set=(X_test, y_test),
              show_metric=True, batch_size=batch_size, snapshot_step=100,
              n_epoch=150)

    # Save it.
    model.save('checkpoints/rnn.tflearn')

In [3]:
def test_main(filename, frames, batch_size, num_classes, input_length):
    """From the blog post linked above."""
    # Get our data.
    X_train, _, y_train, _ = get_data(filename, frames, num_classes, input_length)
    
    # Get sizes.
    num_classes = len(y_train[0])

    # Get our network.
    tf.reset_default_graph()
    net = get_network_wide(frames, input_length, num_classes)

    # Get our model.
    model = tflearn.DNN(net, tensorboard_verbose=0)
    model.load('checkpoints/rnn.tflearn', weights_only = True)

    # Evaluate.
    print(model.evaluate(X_train, y_train))

In [4]:
def model_train():
    
    filename = 'cnn-features.pkl'
    input_length = 2048
    
    # If we were sending the predicted classes instead of the CNN features
    # filename = 'test-data-predicted-frames.pkl'
    # input_length = 2
    
    # Since RNN takes in a vector of features, here we have taken the vector size to be 10 - gave the best score
    frames = 10
    
    # All the images cannot be fed into the neural network at once, hence created batches - took a random number ( ideally 32 is chosen )
    batch_size = 32
    
    # Number of labels - to try the model, had considered 10 classes intially
    num_classes = 10

    train_main(filename, frames, batch_size, num_classes, input_length)


In [5]:
def model_test():
    # If we were sending the predicted classes instead of the CNN features
    # filename = 'data/predicted-frames-2.pkl'
    # input_length = 2
    
    filename = 'test-data-cnn-features.pkl'
    input_length = 2048
    frames = 10
    batch_size = 32
    num_classes = 10

    test_main(filename, frames, batch_size, num_classes, input_length)


In [7]:
# Test Set 2
if __name__ == '__main__':
#   model_train()
    model_test()

Total dataset size: 235
(235, 10, 1, 1, 1, 2048)
(235,)
INFO:tensorflow:Restoring parameters from /Users/pkaur/tensorflow/SignLanguage/checkpoints/rnn.tflearn
[0.57345970405786528]


In [8]:
# Test Set 3
if __name__ == '__main__':
#   model_train()
    model_test()

Total dataset size: 220
(220, 10, 1, 1, 1, 2048)
(220,)
INFO:tensorflow:Restoring parameters from /Users/pkaur/tensorflow/SignLanguage/checkpoints/rnn.tflearn
[0.46969696999800326]


In [9]:
# Test Set 4
if __name__ == '__main__':
#   model_train()
    model_test()

Total dataset size: 213
(213, 10, 1, 1, 1, 2048)
(213,)
INFO:tensorflow:Restoring parameters from /Users/pkaur/tensorflow/SignLanguage/checkpoints/rnn.tflearn
[0.49738220862693189]


In [10]:
# Test Set 1
if __name__ == '__main__':
#   model_train()
    model_test()

EOFError: 